In [1]:
# Entrena un modelo de Gradient Boosted Trees
# para predecir el consumo eléctrico
# y la cantidad de usuarios.

# Tomado y modificado a partir de:
# https://www.datacamp.com/community/tutorials/xgboost-in-python
# https://towardsdatascience.com/a-beginners-guide-to-xgboost-87f5d4c30ed7

import pandas as pd
import numpy as np

import xgboost as xgb

from joblib import dump

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [2]:
path_drive = "/content/drive/MyDrive/Colab/Subsidio_electricidad/" 
path_data = path_drive + "data/Actual/"
fname = "data.csv"

test = []
reg_model = []

data = pd.read_csv(path_data + fname)

data.rename({"Población": "Poblacion"},
    axis = 1, inplace = True)

# Escogemos las variables relevantes.
data.drop(["Area", "CVE_ENT", "CVE_MUN",
    "Tmax", "Tmin", "HDD_mean",
    "HDD_p10",  "CDD_p90", "$luz"],
    axis = 1, inplace = True)

# Unimos las tarifas 1 y DAC.
data["Consumo_1*"] += data["Consumo_DAC"]
data["Usuarios_1*"] += data["Usuarios_DAC"]
data.drop(["Consumo_DAC", "Usuarios_DAC"],
    axis = 1, inplace = True)

# Información utilizada para el modelo.
data.head()

,CVE_INEGI,NOM_ENT,NOM_MUN,Consumo_1*,Usuarios_1*,lon,lat,Tmean,Pre,CDD_mean,Pre_Tmean,Poblacion,PIB,Densidad_poblacion,PCI,$GLP,Año
0,1001,Aguascalientes,Aguascalientes,302560818.0,235978.0,-102.295872,21.811400,19.460571,-661.522583,-661.522583,400.161505,797010.0,1.170000e+11,6.833315,146798.659992,10.151,2010
1,1002,Aguascalientes,Asientos,12805524.0,10624.0,-102.045590,22.126479,18.816061,-678.113220,-678.113220,302.348158,45492.0,2.970839e+09,0.837213,65304.649499,10.151,2010
2,1003,Aguascalientes,Calvillo,17578788.0,15496.0,-102.704911,21.900640,19.020189,-624.275757,-624.275757,446.077944,54136.0,3.708914e+09,0.586295,68511.043575,10.151,2010
3,1004,Aguascalientes,CosÃ­o,4015852.0,3529.0,-102.297038,22.360619,18.941561,-690.588013,-690.588013,316.695232,15042.0,1.233468e+09,1.171645,82001.610624,10.151,2010
4,1005,Aguascalientes,JesÃºs MarÃ­a,31208047.0,26103.0,-102.445700,21.932112,19.160786,-633.728210,-633.728210,378.999951,99590.0,1.121882e+10,1.992218,112650.094989,10.151,2010


In [ ]:
# Estadísticos de las variables.
data.describe()

,Consumo_1*,Usuarios_1*,lon,lat,T_mean,CDD_mean,Pre,Pre_Tmean,Densidad_población,PCI,$GLP,Población,PIB,Año,CVE_INEGI
count,1.728300e+04,17283.000000,17283.000000,17283.000000,17283.000000,17283.000000,17283.000000,17283.000000,17283.000000,1.728300e+04,17283.000000,1.728300e+04,1.728300e+04,17283.000000,17283.000000
mean,2.156270e+07,13456.785049,-98.743039,20.022814,22.427123,496.434783,1106.790031,836.905261,2.879387,8.464168e+04,10.993159,4.726342e+04,6.765280e+09,2013.000000,19324.164844
std,8.631740e+07,39616.449058,4.384691,3.345834,3.377748,217.887584,633.619129,468.075952,11.834312,7.166418e+04,0.851153,1.360173e+05,2.694315e+10,2.000058,7381.610743
min,2.398000e+04,63.000000,-116.889017,14.639368,14.419035,125.732559,0.000000,0.000000,0.001441,1.273697e+04,8.948300,8.600000e+01,2.373494e+06,2010.000000,1001.000000
25%,1.043140e+06,1163.000000,-100.737771,17.620009,19.967443,358.526184,682.422936,526.375082,0.191171,4.303953e+04,10.238682,4.274000e+03,2.403014e+08,2011.000000,14079.000000
50%,3.224329e+06,3308.000000,-98.236805,19.329310,21.914389,434.459412,941.986782,723.676950,0.527785,6.653278e+04,11.104057,1.301700e+04,8.416350e+08,2013.000000,20226.000000
75%,1.008426e+07,8869.000000,-96.769306,20.934726,24.924725,545.298798,1387.195159,1019.995059,1.374554,9.751916e+04,11.637004,3.362050e+04,2.806443e+09,2015.000000,24027.000000
max,2.173237e+09,537946.000000,-86.908934,32.434882,31.648048,1637.347656,5187.519554,3528.052812,176.561449,1.234769e+06,13.169542,1.830284e+06,4.900000e+11,2016.000000,32058.000000


In [ ]:
# Matriz de correlación de Pearson.
data.corr()

,Consumo_1*,Usuarios_1*,lon,lat,T_mean,CDD_mean,Pre,Pre_Tmean,Densidad_población,PCI,$GLP,Población,PIB,Año,CVE_INEGI
Consumo_1*,1.000000,0.844769,-1.517104e-01,1.966613e-01,0.037609,0.175000,-0.108148,-0.093338,0.318539,0.280889,0.005908,0.807671,0.749397,1.314590e-02,-6.607402e-02
Usuarios_1*,0.844769,1.000000,-1.204962e-01,1.351577e-01,-0.047144,0.095912,-0.112452,-0.082853,0.541620,0.309517,0.017101,0.985200,0.889541,1.865308e-02,-1.208885e-01
lon,-0.151710,-0.120496,1.000000e+00,-6.648263e-01,0.368563,-0.570965,0.487079,0.412035,-0.000243,-0.270591,-0.013115,-0.111694,-0.085190,5.813482e-18,2.369749e-01
lat,0.196661,0.135158,-6.648263e-01,1.000000e+00,-0.172404,0.901895,-0.477210,-0.406219,-0.029756,0.368796,0.037059,0.121039,0.110049,1.078267e-17,-1.816479e-02
T_mean,0.037609,-0.047144,3.685626e-01,-1.724041e-01,1.000000,0.002264,0.406738,0.400219,-0.174850,-0.001128,0.025122,-0.066071,-0.048065,4.720407e-02,1.749890e-01
CDD_mean,0.175000,0.095912,-5.709655e-01,9.018954e-01,0.002264,1.000000,-0.402018,-0.337884,-0.071880,0.331921,0.036149,0.078861,0.075153,7.924268e-02,-4.882388e-02
Pre,-0.108148,-0.112452,4.870794e-01,-4.772099e-01,0.406738,-0.402018,1.000000,0.921505,-0.071052,-0.257785,-0.093505,-0.105271,-0.096163,-6.036922e-02,1.105634e-01
Pre_Tmean,-0.093338,-0.082853,4.120352e-01,-4.062194e-01,0.400219,-0.337884,0.921505,1.000000,-0.049735,-0.196991,-0.037105,-0.075536,-0.076816,-1.245438e-03,1.161193e-01
Densidad_población,0.318539,0.541620,-2.431198e-04,-2.975554e-02,-0.174850,-0.071880,-0.071052,-0.049735,1.000000,0.229053,0.003081,0.560745,0.585140,5.724501e-03,-7.796424e-02
PCI,0.280889,0.309517,-2.705908e-01,3.687962e-01,-0.001128,0.331921,-0.257785,-0.196991,0.229053,1.000000,0.066342,0.283659,0.473212,4.605540e-02,-1.489635e-01


In [12]:
# Predicción de usuarios.

# Escogemos el conjunto de features y de variables a predecir.
X, Y = data.iloc[:,6:], data.iloc[:,3]

# Quitamos algunas features.
X.drop(["Pre", "PCI", "Densidad_población"],
    axis = 1, inplace = True)

# Separamos en conjuntos de entrenamiento y de prueba.
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size = 0.3 )

# Separamos en conjunto de prueba y de validación.
X_train, X_val, Y_train, Y_val = train_test_split(
    X, Y, test_size = 0.5 )

In [9]:
# Features a utilizar.
X.head()

,T_mean,CDD_mean,Pre_Tmean,$GLP,Poblacion,PIB,Año,CVE_INEGI
0,19.460571,517.162109,400.161505,10.151,797010.0,1.170000e+11,2010,1001
1,18.816061,562.153564,302.348158,10.151,45492.0,2.970839e+09,2010,1002
2,19.020189,464.146240,446.077944,10.151,54136.0,3.708914e+09,2010,1003
3,18.941561,577.571777,316.695232,10.151,15042.0,1.233468e+09,2010,1004
4,19.160786,487.516479,378.999951,10.151,99590.0,1.121882e+10,2010,1005


In [10]:
# Variable a predecir.
Y.head()

0    235978.0
1     10624.0
2     15496.0
3      3529.0
4     26103.0
Name: Usuarios_1*, dtype: float64

In [13]:
# Regresión lineal.

# Copiamos las variables para poder
# transformarlas en caso de ser necesario.
X_reg = X_train.copy()
Y_reg = Y_train.copy()
X_reg_val = X_val.copy()
X_reg_test = X_test.copy()

# Creamos el regresor.
lin_reg = LinearRegression()

# Entrenamos el modelo.
lin_reg.fit(X_reg, Y_reg)

# Probamos la regresión en el set de entrenamiento.
preds = lin_reg.predict(X_reg)

# Calculamos el error de entrenamiento.
rmse = np.sqrt(mean_squared_error(Y_reg, preds))
r2 = r2_score(Y_reg, preds)

print("Entrenamiento")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

Entrenamiento
RMSE: 6.995E+03
R^2:  0.9718247


In [14]:
# Probamos la regresión en el set de validación.
preds = lin_reg.predict(X_reg_val)

# Calculamos el error de validación.
rmse = np.sqrt(mean_squared_error(Y_val, preds))
r2 = r2_score(Y_val, preds)

print("Validación")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

# Probamos la regresión en el set de prueba
# y guardamos el resultado para su evaluación final.
test_0 = lin_reg.predict(X_reg_test)

Validación
RMSE: 5.910E+03
R^2:  0.9750823


In [17]:
# Calculamos el error de prueba.
rmse = np.sqrt(mean_squared_error(Y_test, test_0))
r2 = r2_score(Y_test, test_0)

print("Prueba")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

# Guardamos el modelo entrenado.
dump(lin_reg, path_drive + "reg_model_usuarios.joblib")

Prueba
RMSE: 6.965E+03
R^2:  0.9703591


['/content/drive/MyDrive/Colab/Subsidio_electricidad/reg_model_usuarios.joblib']

In [ ]:
# Predicción de consumo.
# Escogemos el conjunto de features y de variables a predecir.
X, Y = data.iloc[:,6:], data.iloc[:,2]

# Quitamos algunas features.
X.drop(["Pre", "PCI", "Densidad_población"],
    axis = 1, inplace = True)

# Separamos en conjuntos de entrenamiento y de prueba.
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size = 0.3 )

# Separamos en conjunto de prueba y de validación.
X_train, X_val, Y_train, Y_val = train_test_split(
    X, Y, test_size = 0.5 )

In [ ]:
# Variable a predecir.
Y.head()

0    302560818.0
1     12805524.0
2     17578788.0
3      4015852.0
4     31208047.0
Name: Consumo_1*, dtype: float64

In [ ]:
# Método 1.
# Regresión lineal.

# Copiamos las variables para poder
# transformarlas en caso de ser necesario.
X_reg = X_train.copy()
Y_reg = Y_train.copy()
X_reg_val = X_val.copy()
X_reg_test = X_test.copy()

# Creamos el regresor.
lin_reg_1 = LinearRegression()

# Entrenamos el modelo.
lin_reg_1.fit(X_reg, Y_reg)

# Probamos la regresión en el set de entrenamiento.
preds = lin_reg_1.predict(X_reg)

# Calculamos el error de entrenamiento.
rmse = np.sqrt(mean_squared_error(Y_reg, preds))
r2 = r2_score(Y_reg, preds)

print("Entrenamiento")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

Entrenamiento
RMSE: 4.933E+07
R^2:  0.6552070


In [ ]:
# Probamos la regresión en el set de validación.
preds = lin_reg_1.predict(X_reg_val)

# Calculamos el error de validación.
rmse = np.sqrt(mean_squared_error(Y_val, preds))
r2 = r2_score(Y_val, preds)

print("Validación")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

# Probamos la regresión en el set de prueba
# y guardamos el resultado para su evaluación final.
test_1 = lin_reg_1.predict(X_reg_test)

test.append(test_1)
reg_model.append(lin_reg_1)

Validación
RMSE: 4.821E+07
R^2:  0.7036956


In [ ]:
# Método 2.
# Regresión lineal con transformación logarítmica.

# Copiamos las variables para poder
# transformarlas en caso de ser necesario.
X_reg = X.copy()
Y_reg = Y.copy()
X_reg_val = X_val.copy()
X_reg_test = X_test.copy()

# Sacamos el logaritmo.
X_reg[["Poblacion", "PIB"]] = np.log(
    X_reg[["Poblacion", "PIB"]])
Y_reg = np.log(Y_reg)
X_reg_val[["Poblacion", "PIB"]] = np.log(
    X_reg_val[["Poblacion", "PIB"]])
X_reg_test[["Poblacion", "PIB"]] = np.log(
    X_reg_test[["Poblacion", "PIB"]])

# Creamos el regresor.
lin_reg_2 = LinearRegression()

# Entrenamos el modelo.
lin_reg_2.fit(X_reg, Y_reg)

# Probamos la regresión en el set de entrenamiento.
preds = lin_reg_2.predict(X_reg)

# Quitamos el logaritmo para evaluar el error.
Y_reg = np.exp(Y_reg)
preds = np.exp(preds)

# Calculamos el error de entrenamiento.
rmse = np.sqrt(mean_squared_error(Y_reg, preds))
r2 = r2_score(Y_reg, preds)

print("Entrenamiento")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

Entrenamiento
RMSE: 3.819E+07
R^2:  0.8042005


In [ ]:
# Probamos la regresión en el set de validación.
preds = lin_reg_2.predict(X_reg_val)

# Quitamos el logaritmo para evaluar el error.
preds = np.exp(preds)

# Calculamos el error de validación.
rmse = np.sqrt(mean_squared_error(Y_val, preds))
r2 = r2_score(Y_val, preds)

print("Validación")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

# Probamos la regresión en el set de prueba
# y guardamos el resultado para su evaluación final.
test_2 = lin_reg_2.predict(X_reg_test)
test_2 = np.exp(test_2)

test.append(test_2)
reg_model.append(lin_reg_2)

Validación
RMSE: 3.757E+07
R^2:  0.8200393


In [ ]:
# Método 3.
# Entrenamiento simple.

# Copiamos las variables para poder
# transformarlas en caso de ser necesario.
X_reg = X_train.copy()
Y_reg = Y_train.copy()
X_reg_val = X_val.copy()
X_reg_test = X_test.copy()

# Hiperparámetros.
params = {
    "objective": "reg:squarederror",
    "colsample_bytree": 0.3,
    "learning_rate": 0.1,
    "max_depth": 50,
    "min_child_weight" : 25,
    "min_split_loss": 0.3,
    "n_estimators": 100
    }

# Creamos el regresor con los hiperparámetros.
xg_reg_3 = xgb.XGBRegressor( **params )

# Entrenamos el modelo.
xg_reg_3.fit(X_reg, Y_reg, verbose = True)

# Probamos la regresión en el set de entrenamiento.
preds = xg_reg_3.predict(X_reg)

# Calculamos el error de entrenamiento.
rmse = np.sqrt(mean_squared_error(Y_reg, preds))
r2 = r2_score(Y_reg, preds)

print("Entrenamiento")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

Entrenamiento
RMSE: 1.880E+07
R^2:  0.9499345


In [ ]:
# Probamos la regresión en el set de validación.
preds = xg_reg_3.predict(X_reg_val)

# Calculamos el error de validación.
rmse = np.sqrt(mean_squared_error(Y_val, preds))
r2 = r2_score(Y_val, preds)

print("Validación")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

# Probamos la regresión en el set de prueba
# y guardamos el resultado para su evaluación final.
test_3 = xg_reg_3.predict(X_reg_test)

test.append(test_3)
reg_model.append(xg_reg_3)

Validación
RMSE: 2.850E+07
R^2:  0.8964479


In [ ]:
# Método 4.
# Entrenamiento simple con
# transformación logarítmica.

# Copiamos las variables para poder
# transformarlas en caso de ser necesario.
X_reg = X_train.copy()
Y_reg = Y_train.copy()
X_reg_val = X_val.copy()
X_reg_test = X_test.copy()

# Sacamos el logaritmo.
X_reg[["Poblacion", "PIB"]] = np.log(
    X_reg[["Poblacion", "PIB"]])
Y_reg = np.log(Y_reg)
X_reg_val[["Poblacion", "PIB"]] = np.log(
    X_reg_val[["Poblacion", "PIB"]])
X_reg_test[["Poblacion", "PIB"]] = np.log(
    X_reg_test[["Poblacion", "PIB"]])

# Hiperparámetros.
params = {
    "objective": "reg:squarederror",
    "colsample_bytree": 0.3,
    "learning_rate": 0.1,
    "max_depth": 50,
    "min_child_weight" : 25,
    "min_split_loss": 0.3,
    "n_estimators": 100
    }

# Creamos el regresor con los hiperparámetros.
xg_reg_4 = xgb.XGBRegressor( **params )

# Entrenamos el modelo.
xg_reg_4.fit(X_reg, Y_reg, verbose = True)

# Probamos la regresión en el set de entrenamiento.
preds = xg_reg_4.predict(X_reg)

# Quitamos el logaritmo para evaluar el error.
Y_reg = np.exp(Y_reg)
preds = np.exp(preds)

# Calculamos el error de entrenamiento.
rmse = np.sqrt(mean_squared_error(Y_reg, preds))
r2 = r2_score(Y_reg, preds)

print("Entrenamiento")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

Entrenamiento
RMSE: 1.709E+07
R^2:  0.9586014


In [ ]:
# Probamos la regresión en el set de validación.
preds = xg_reg_4.predict(X_reg_val)

# Quitamos el logaritmo para evaluar el error.
preds = np.exp(preds)

# Calculamos el error de validación.
rmse = np.sqrt(mean_squared_error(Y_val, preds))
r2 = r2_score(Y_val, preds)

print("Validación")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

# Probamos la regresión en el set de prueba
# y guardamos el resultado para su evaluación final.
test_4 = xg_reg_4.predict(X_reg_test)
test_4 = np.exp(test_4)

test.append(test_4)
reg_model.append(xg_reg_4)

Validación
RMSE: 2.391E+07
R^2:  0.9271308


In [ ]:
# Método 5.
# Cross-Validation.

# Copiamos las variables para poder
# transformarlas en caso de ser necesario.
X_reg = X_train.copy()
Y_reg = Y_train.copy()
X_reg_val = X_val.copy()
X_reg_test = X_test.copy()

# Sacamos el logaritmo.
X_reg[["Poblacion", "PIB"]] = np.log(
    X_reg[["Poblacion", "PIB"]])
Y_reg = np.log(Y_reg)
X_reg_val[["Poblacion", "PIB"]] = np.log(
    X_reg_val[["Poblacion", "PIB"]])
X_reg_test[["Poblacion", "PIB"]] = np.log(
    X_reg_test[["Poblacion", "PIB"]])

# Hiperparámetros.
params = {
    "colsample_bytree": [ 0.3 ],
    "learning_rate": [ 0.1 ],
    "max_depth": [ 50 ],
    "min_child_weight" : [ 25 ],
    "min_split_loss": [ 0.3 ],
    "n_estimators": [ 100 ]
    }

# Creamos el regresor.
xg_reg = xgb.XGBRegressor(
    objective = "reg:squarederror"
    )

# Ajustador de hiperparámetros con Cross Validation.
grid_5 = GridSearchCV(xg_reg, params,
    n_jobs = -1, verbose = 2,
    scoring = "neg_mean_absolute_error")

# Entrenamos el modelo.
grid_5.fit(X_reg, Y_reg)

# Probamos la regresión en el set de entrenamiento.
preds = grid_5.predict(X_reg)

# Calculamos el error de entrenamiento.
rmse = np.sqrt(mean_squared_error(Y_reg, preds))
r2 = r2_score(Y_reg, preds)

print("Entrenamiento")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    5.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    5.2s finished


Entrenamiento
RMSE: 2.166E-01
R^2:  0.9854706


In [ ]:
# Probamos la regresión en el set de validación.
preds = grid_5.predict(X_reg_val)

# Quitamos el logaritmo para evaluar el error.
preds = np.exp(preds)

# Calculamos el error de validación.
rmse = np.sqrt(mean_squared_error(Y_val, preds))
r2 = r2_score(Y_val, preds)

print("Validación")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

# Probamos la regresión en el set de prueba
# y guardamos el resultado para su evaluación final.
test_5 = grid_5.predict(X_reg_test)
test_5 = np.exp(test_5)

test.append(test_5)
reg_model.append(grid_5)

Validación
RMSE: 2.391E+07
R^2:  0.9271308


In [ ]:
# Método 6.
# Ajuste de hiperparámetros.

# Tarda bastante en ejecutar, usar
# como base para los hiperparámetros
# en otros métodos.
'''
# Copiamos las variables para poder
# transformarlas en caso de ser necesario.
X_reg = X_train.copy()
Y_reg = Y_train.copy()
X_reg_val = X_val.copy()
X_reg_test = X_test.copy()

# Sacamos el logaritmo.
X_reg[["Poblacion", "PIB"]] = np.log(
    X_reg[["Poblacion", "PIB"]])
Y_reg = np.log(Y_reg)
X_reg_val[["Poblacion", "PIB"]] = np.log(
    X_reg_val[["Poblacion", "PIB"]])
X_reg_test[["Poblacion", "PIB"]] = np.log(
    X_reg_test[["Poblacion", "PIB"]])

# Hiperparámetros
params = {
    "colsample_bytree": [ 0.3 ],
    "learning_rate": [ 0.1 ],
    "learning_rate": [ 0.1, 0.2, 0.3, 0.4, 0.5 ],
    "max_depth": [ 50 ],
    "min_child_weight" : [ 0, 10, 15, 20, 25 ],
    "min_split_loss": [ 0.0, 0.3, 0.6, 0.9, 1.2 ],
    #"alpha": [ 0, 3, 6, 9, 12 ],
    #"lambda": [ 0, 3, 6, 9, 12 ],
    "n_estimators": [ 100 ],
    }

# Creamos el regresor.
xg_reg = xgb.XGBRegressor(
    objective = "reg:squarederror")

# Ajustador de hiperparámetros con Cross Validation.
grid_6 = GridSearchCV(xg_reg, params,
    n_jobs = -1, cv = 2, verbose = 2,
    scoring = "neg_mean_absolute_error")

# Entrenamos el modelo.
grid_6.fit(X_reg, Y_reg)

# Probamos la regresión en el set de entrenamiento.
preds = grid_6.predict(X_reg)

# Calculamos el error de entrenamiento.
rmse = np.sqrt(mean_squared_error(Y_reg, preds))
r2 = r2_score(Y_reg, preds)

print("Entrenamiento")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")
print("Mejores parámetros")
print(grid_6.best_params_)
'''

Fitting 2 folds for each of 125 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   20.9s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  2.7min finished


Entrenamiento
RMSE: 2.166E-01
R^2:  0.9854706
Mejores parámetros
{'colsample_bytree': 0.3, 'learning_rate': 0.1, 'max_depth': 50, 'min_child_weight': 25, 'min_split_loss': 0.3, 'n_estimators': 100}


In [ ]:
# Probamos la regresión en el set de validación.
preds = grid_6.predict(X_reg_val)

# Quitamos el logaritmo para evaluar el error.
preds = np.exp(preds)

# Calculamos el error de validación.
rmse = np.sqrt(mean_squared_error(Y_val, preds))
r2 = r2_score(Y_val, preds)

print("Validación")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

# Probamos la regresión en el set de prueba
# y guardamos el resultado para su evaluación final.
test_6 = grid_6.predict(X_reg_test)
test_6 = np.exp(test_6)

test.append(test_6)
reg_model.append(grid_6)

Validación
RMSE: 2.391E+07
R^2:  0.9271308


In [ ]:
i = 4

# Calculamos el error de prueba.
rmse = np.sqrt(mean_squared_error(Y_test, test[i - 1]))
r2 = r2_score(Y_test, test[i - 1])

print("Prueba")
print(f"Método {i}")
print(f"RMSE: {rmse:.3E}")
print(f"R^2:  {r2:.7f}")

# Guardamos el modelo entrenado.
dump(reg_model[i - 1], path_drive + "reg_model_consumo.joblib")

Prueba
Método 4
RMSE: 1.927E+07
R^2:  0.9408597


['/content/drive/MyDrive/Colab/reg_model.joblib']